In [2]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from pathlib import Path
from collections import namedtuple
import torchvision.transforms as T
import torch.nn.functional as F
import time

In [2]:
state = torch.load('best.pth')

In [3]:
state.keys()

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'accuracy', 'loss'])

In [4]:
net = torchvision.models.densenet201(num_classes=2)

In [5]:
net.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [6]:
# net

In [7]:
# state['optimizer_state_dict']

In [3]:
net = torch.load('model_211206.pt')

In [4]:
folder = r'/dataset/kerrytj/kerrytj-image/image'



In [5]:
folder_path = Path(folder)

In [6]:
dirs = sorted(list(folder_path.iterdir()))
print(dirs)
classes = [i.name for i in dirs]
print(classes)

[PosixPath('/dataset/kerrytj/kerrytj-image/image/bad'), PosixPath('/dataset/kerrytj/kerrytj-image/image/good')]
['bad', 'good']


In [7]:
image_paths = [sorted(list(i.iterdir())) for i in dirs]

In [8]:
# image_paths

In [9]:
# Prediction = namedtuple('Prediction', ['path', 'target', 'predicted'])

class Prediction:
    def __init__(self, path, target, predicted, conf):
        self.path = path
        self.target = target
        self.predicted = predicted
        self.conf = conf
        
    def __repr__(self):
        return str((self.path, self.target, self.predicted, self.conf))

In [10]:
predictions = [ Prediction(p, i, None, None) for i, paths in enumerate(image_paths) for p in paths]

In [11]:
# predictions


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('use device', device)

use device cuda


In [13]:
net = net.eval().to(device)

In [14]:
transform = T.Compose([
    T.Resize((540, 540)),
    T.ToTensor()
])

In [20]:

for p in predictions:
    with Image.open(p.path) as im:
        input_tensor = transform(im)
    input_tensor = input_tensor.unsqueeze(dim=0)
    input_tensor = input_tensor.to(device)
    
    with torch.no_grad():
        result = net(input_tensor)
        result = F.softmax(result, dim=1)
    result = result.squeeze(dim=0)
    conf, index = result.max(dim=0)
    index = index.cpu().item()
    conf = conf.cpu().item()
    
    p.predicted = index
    p.conf = conf
    


In [21]:
for p in predictions:
    if p.target != p.predicted:
        print(p)

(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/082407_98204676247_S.JPG'), 0, 1, 0.9982408285140991)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/091717_74162950733_S.JPG'), 0, 1, 0.9737665057182312)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/092405_43666481794_S.JPG'), 0, 1, 0.6395330429077148)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/100333_6239074446_S.JPG'), 0, 1, 0.8435037136077881)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/103905_40350653161_S.JPG'), 0, 1, 0.6401761174201965)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/104935_90716376109_S.JPG'), 0, 1, 0.9015095829963684)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/134603_15884762824_S.JPG'), 0, 1, 0.9847902655601501)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/092027_74165340169_S.JPG'), 1, 0, 0.7600827217102051)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/123304_40385143393_S.JPG'), 1, 0, 0.8958895206451416)
(PosixPath('/dataset/kerrytj/kerrytj

In [22]:
miss_count = len(list(filter(lambda p: p.target != p.predicted, predictions)))
print(1 - miss_count / len(predictions))

0.989065606361829


In [48]:
transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [49]:
def do_prediction(net, transform):
    predictions = [ Prediction(p, i, None, None) for i, paths in enumerate(image_paths) for p in paths]
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # device = torch.device('cpu')
    print('use device', device)
    net = net.eval().to(device)
    
    for p in predictions:
        with Image.open(p.path) as im:
            input_tensor = transform(im)
        input_tensor = input_tensor.unsqueeze(dim=0)
        input_tensor = input_tensor.to(device)

        with torch.no_grad():
            result = net(input_tensor)
            result = F.softmax(result, dim=1)
        result = result.squeeze(dim=0)
        conf, index = result.max(dim=0)
        index = index.cpu().item()
        conf = conf.cpu().item()

        p.predicted = index
        p.conf = conf
        
    return predictions

In [50]:
state = torch.load('best.pth')
net = torchvision.models.densenet201(num_classes=2)
net.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [51]:
predictions = do_prediction(net, transform)

use device cuda


In [52]:
for p in predictions:
    if p.target != p.predicted:
        print(p)

(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/082407_98204676247_S.JPG'), 0, 1, 0.9615916013717651)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/092138_74163202404_S.JPG'), 0, 1, 0.6098570227622986)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/092459_13238508352_S.JPG'), 0, 1, 0.6185823082923889)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/bad/094350_92959832555_S.JPG'), 0, 1, 0.6206309199333191)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/083755_74163467706_S.JPG'), 1, 0, 0.510867178440094)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/085240_43666489548_S.JPG'), 1, 0, 0.8957359194755554)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/091239_93874689744_S.JPG'), 1, 0, 0.9700624346733093)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/091320_44185199326_S.JPG'), 1, 0, 0.5965411067008972)
(PosixPath('/dataset/kerrytj/kerrytj-image/image/good/092027_74165340169_S.JPG'), 1, 0, 0.8810672760009766)
(PosixPath('/dataset/kerrytj/kerr

In [53]:
miss_count = len(list(filter(lambda p: p.target != p.predicted, predictions)))
print(1 - miss_count / len(predictions))

0.963220675944334


In [1]:
123

123

In [26]:
net = torch.load('model_211206.pt')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('use device', device)
net = net.eval().to(device)

use device cuda


In [27]:
transform = T.Compose([
    T.Resize((540, 540)),
    T.ToTensor()
])

In [28]:
time_accu = 0
for p in predictions[:1000]:
    with Image.open(p.path) as im:
        input_tensor = transform(im)
    input_tensor = input_tensor.unsqueeze(dim=0)
    input_tensor = input_tensor.to(device)
    
    start_time = time.time()
    with torch.no_grad():
        result = net(input_tensor)
        result = F.softmax(result, dim=1)
    
    result = result.squeeze(dim=0)
    conf, index = result.max(dim=0)
    index = index.cpu().item()
    conf = conf.cpu().item()
    end_time = time.time()
    
    p.predicted = index
    p.conf = conf
    time_accu += (end_time - start_time)

print(time_accu)

69.41995573043823


In [29]:
state = torch.load('best.pth')
net = torchvision.models.densenet201(num_classes=2)
net.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

In [30]:
net = net.eval().to(device)

In [31]:
transform = T.Compose([
    T.Resize((512, 512)),
    T.ToTensor(),
    T.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [32]:
time_accu = 0
for p in predictions[:1000]:
    with Image.open(p.path) as im:
        input_tensor = transform(im)
    input_tensor = input_tensor.unsqueeze(dim=0)
    input_tensor = input_tensor.to(device)
    
    start_time = time.time()
    with torch.no_grad():
        result = net(input_tensor)
        result = F.softmax(result, dim=1)
    
    result = result.squeeze(dim=0)
    conf, index = result.max(dim=0)
    index = index.cpu().item()
    conf = conf.cpu().item()
    end_time = time.time()
    
    p.predicted = index
    p.conf = conf
    time_accu += (end_time - start_time)

print(time_accu)

269.61998677253723
